In [ ]:
import os
from pymongo import MongoClient
from dotenv import load_dotenv


In [ ]:
load_dotenv()
mongo_uri = os.getenv("mongo")
ord_data_path = os.getenv("fig_rxn_smiles")

client = MongoClient(mongo_uri)
db = client["complement"]
collection = db["fig_rxn_smiles"]

In [ ]:
def read_reactions_from_pbtxt(directory):
    reactions = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".pbtxt"):
                file_path = os.path.join(root, file)
                with open(file_path, "r") as f:
                    reaction = reaction_pb2.Reaction()
                    message_helpers.load_message(f, reaction)
                    reactions.append(reaction)
    return reactions

# Function to convert ORD Reaction to a dictionary format
def reaction_to_dict(reaction):
    # Convert the Protobuf reaction object to a dictionary
    return MessageToDict(reaction)

# Function to export reactions to MongoDB
def export_to_mongodb(reactions):
    for reaction in reactions:
        # Convert each reaction to a dictionary with all fields
        reaction_data = reaction_to_dict(reaction)
        # Insert the reaction into MongoDB
        collection.insert_one(reaction_data)

# Load reactions from the ORD files
reactions = read_reactions_from_pbtxt(ord_data_path)

# Export the reactions to MongoDB
export_to_mongodb(reactions)

print("Export completed successfully with all fields!")
